In [47]:
import requests
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
def research(name_or_id, check):

    # Define the API request parameters
    base_url = "https://api.data.gov/ed/collegescorecard/v1/"
    dataset = "schools.json?"
    fields = ["id",
             "school.name",
             "school.state",
             "school.city",
             "school.school_url",
             "latest.student.size",
             "latest.admissions.admission_rate.overall",
             "latest.completion.completion_rate_4yr_150_asian",
             "latest.earnings.10_yrs_after_entry.median",
             "latest.cost.net_price.private.by_income_level.48001-75000",
             "latest.cost.net_price.private.by_income_level.75001-110000",
             "latest.student.demographics.race_ethnicity.white",
             "latest.student.demographics.race_ethnicity.asian",
             "latest.student.demographics.race_ethnicity.black",
             "latest.student.demographics.race_ethnicity.hispanic",
             "latest.school.ft_faculty_rate",
             "school.locale",
             "school.degree_urbanization",
             "school.carnegie_size_setting",
             "latest.admissions.sat_scores.average.overall",
             "latest.admissions.act_scores.midpoint.cumulative",
             "latest.admissions.test_requirements",
             #"latest.programs.1107.earnings.highest.3_yr.not_enrolled.overall_count",
             "oops.variables.does.not.exist"]
    field  = ["id",
             "Name",
             "State",
             "City",
             "Web site",
             "Student Size",
             "Admission Rate",
             "Graduation Rate (Asian)",
             "Earning after 10yr",
             "Cost(48000-75000)",
             "Cost(75000-110000)",
             "White",
             "Asian",
             "Black",
             "Hispanic",
             "%Full Time Faculty",
             "locale",
             "locale_num",
             "carnegie size setting",
             "SAT average",
             "ACT midpoint cumulative",
             #"Earning",
             "TestOptional"]
    options = "&per_page=100&page=0"
    api_key = "&api_key=LaxSQBL0fIb0ia1qK6gmKlIXfk4FpcxNUCzF1WV1"
    

    def use_id(college_id):
        url = f'{base_url}schools?_fields={",".join(fields)}&id={college_id}{options}{api_key}'
        return url

    def use_name(college_name):
        url = f'{base_url}schools?school.name={college_name}&fields={"%2C".join(fields)}{options}{api_key}'
        return url
    
    if type(name_or_id) == str:
        #Use name to search
        request_url = use_name(name_or_id)
    
    elif type(name_or_id) == int:
        #Use id to search
        request_url = use_id(name_or_id)
    else:
        return 0


    # Make the API request and create a DataFrame
    response = requests.get(request_url)
    #print(response.json())

    college_info = pd.DataFrame(response.json()["results"])
    df = pd.DataFrame(response.json()["results"])
    
    #test optional?            
    def test_optional():   
        test = int(df["latest.admissions.test_requirements"])
        if test == 1:
            req="required"
        elif test == 2:
            req="recommended"
        elif test == 3:
            req="neither"
        elif test == 4:
            req="don't know"
        elif test == 5:
            req="considered but not required"
        df["latest.admissions.test_requirements"] = req 
        print(df["latest.admissions.test_requirements"])
    
    #Carnegie Size 
    def carnegie_size():
        carnegie_size_setting = int(df["school.carnegie_size_setting"])
        carnegie=["Not classified",
                  "2y, XS",
                  "2y, S",
                  "2y, M",
                  "2y, L",
                  "2y, XL",
                  "4y, XS, primarily nonresidential",
                  "4y, XS, primarily residential",
                  "4y, XS, highly residential",
                  "4y, S, primarily nonresidential",
                  "4y, S, primarily residential",
                  "4y, S, highly residential",
                  "4y, M, primarily nonresidential",
                  "4y, M, primarily residential",
                  "4y, M, highly residential",
                  "4y, L, primarily nonresidential",
                  "4y, L, primarily residential",
                  "4y, L, highly residential",
                  "Exclusively graduate/professional"]
        df["school.carnegie_size_setting"] = carnegie[carnegie_size_setting]
    
    def locale():
        locale = int(df["school.locale"])
        locale_level = [["City: Large", "City: Midsize", "City: Small"],
                        ["Suburb: Large", "Suburb: Midsize", "Suburb: Small"],
                        ["Town: Fringe", "Town: Distant", "Town: Remote"],
                        ["Rural: Fringe", "Rural: Distant", "Rural: Remote"]]
        locale_num = [[1, 2, 3],
                      [4, 5, 6],
                      [7, 8, 9], 
                      [10, 11, 12]]
        df["school.locale"] = locale_level[locale//10 - 1][locale%10 - 1]
        df["school.degree_urbanization"] = locale_num[locale//10 - 1][locale%10 - 1]
    
    if check:
        if not df["latest.admissions.test_requirements"].empty:
            test_optional()
        if not df["school.carnegie_size_setting"].empty:
            carnegie_size()
        if not df["school.locale"].empty:
            locale()

    # Set up the Google Sheets API credentials
    scope = ['https://spreadsheets.google.com/feeds',
             'https://www.googleapis.com/auth/drive']
    creds = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    client = gspread.authorize(creds)

    # Open the existing spreadsheet and select the shee
    sheet_name = 'Research Universities'
    #sheet_name = "Liberal Arts"
    spreadsheet_url = 'https://docs.google.com/spreadsheets/d/17ktsMq2LboE5q_9nm4MLFnVN59pffnpS4HC0UgCrj34/edit#gid=0'
    spreadsheet = client.open_by_url(spreadsheet_url)
    worksheet = spreadsheet.worksheet(sheet_name)

    # Get the index of the last row with data and start appending from the next row
    next_row = len(worksheet.get_all_values()) + 1


    new_order = ["id",
             "school.name",
             "school.state",
             "school.city",
             "school.school_url",
             "latest.student.size",
             "latest.admissions.admission_rate.overall",
             "latest.completion.completion_rate_4yr_150_asian",
             "latest.earnings.10_yrs_after_entry.median",
             "latest.cost.net_price.private.by_income_level.48001-75000",
             "latest.cost.net_price.private.by_income_level.75001-110000",
             "latest.student.demographics.race_ethnicity.white",
             "latest.student.demographics.race_ethnicity.asian",
             "latest.student.demographics.race_ethnicity.black",
             "latest.student.demographics.race_ethnicity.hispanic",
             "latest.school.ft_faculty_rate",
             "school.locale",
             "school.degree_urbanization",
             "school.carnegie_size_setting",
             "latest.admissions.sat_scores.average.overall",
             "latest.admissions.act_scores.midpoint.cumulative",
             #"latest.programs.1101.earnings.highest.3_yr.not_enrolled.overall_count",
             "latest.admissions.test_requirements"]
    
    #print(df)

    new_df = df.loc[:, new_order]
    # Rename the columns in the new DataFrame to match the field names
    new_df.columns = field

    # Loop through each row and append the data to the sheet
    for i, row in new_df.iterrows():
        row_data = []
        for column in field:
            value = row[column] if column in row.index else ""
            row_data.append(str(value))
        worksheet.append_row(row_data, value_input_option='USER_ENTERED')
    print("success")


In [50]:
list = [147767,
179867,
227757,
139755,
131496]
for i in (list):
    research(i, True)

0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    required
Name: latest.admissions.test_requirements, dtype: object
success
0    required
Name: latest.admissions.test_requirements, dtype: object
success


In [43]:
list = [168342,
198385,
168218,
164465,
212009,
230959,
216287,
121345,
130697,
212911,
234207,
161004,
153384,
161086]
for i in (list):
    research(i,True)

0    neither
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    recommended
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions.test_requirements, dtype: object
success
0    considered but not required
Name: latest.admissions

In [167]:
import requests
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
def research(name_or_id):

    # Define the API request parameters
    base_url = "https://api.data.gov/ed/collegescorecard/v1/"
    dataset = "schools.json?"
    fields = ["school.name",
             "latest.programs.cip_4_digit.earnings.highest.3_yr.overall_median_earnings",
             "latest.programs.cip_4_digit.title",
             "oops.variables.does.not.exist"]
    field  = ["Name",
             "earnings",
             "title"]
    options = "&per_page=100&page=0"
    api_key = "&api_key=LaxSQBL0fIb0ia1qK6gmKlIXfk4FpcxNUCzF1WV1"
    

    def use_id(college_id):
        url = f'{base_url}schools?_fields={",".join(fields)}&id={college_id}{options}{api_key}'
        print(url)
        return url

    def use_name(college_name):
        url = f'{base_url}schools?school.name={college_name}&fields={"%2C".join(fields)}{options}{api_key}'
        print(url)
        return url
    
    if type(name_or_id) == str:
        #Use name to search
        request_url = use_name(name_or_id)
    
    elif type(name_or_id) == int:
        #Use id to search
        request_url = use_id(name_or_id)
    else:
        return 0
    

    # Make the API request and create a DataFrame
    response = requests.get(request_url)
    #print(response.json())
    a = response.json()["results"][0]["latest.programs.cip_4_digit"]
    print(len(a))
    for i in range(len(a)):
        b = a[i]["title"]
        if b == "Computer Science." or b == "Neurobiology and Neurosciences.":
            c = a[i]["earnings"]["highest"]["3_yr"]["overall_median_earnings"]
            print(b, c)

    
    college_info = pd.DataFrame(response.json()["results"])
    print(type(college_info))
    #print(college_info.at["Computer Science", "earnings"])
    df = pd.DataFrame(response.json()["results"])

In [171]:
research(227757)

https://api.data.gov/ed/collegescorecard/v1/schools?_fields=school.name,latest.programs.cip_4_digit.earnings.highest.3_yr.overall_median_earnings,latest.programs.cip_4_digit.title,oops.variables.does.not.exist&id=227757&per_page=100&page=0&api_key=LaxSQBL0fIb0ia1qK6gmKlIXfk4FpcxNUCzF1WV1
121
Neurobiology and Neurosciences. None
<class 'pandas.core.frame.DataFrame'>


In [13]:
list = [[0,2],[4,8]]

In [17]:
list[1][0]

4